In [1]:
from konlpy.tag import Okt

okt = Okt()

In [2]:
okt.morphs('한글 형태소 분석기(오케이티)로 테스트 해보았습니다 정상 설치 및 동작이 잘 됩니다.')

['한글',
 '형태소',
 '분석',
 '기',
 '(',
 '오',
 '케이티',
 ')',
 '로',
 '테스트',
 '해보았습니다',
 '정상',
 '설치',
 '및',
 '동작',
 '이',
 '잘',
 '됩니다',
 '.']

In [5]:
from eunjeon import Mecab

In [7]:
mecab = Mecab() 
mecab.pos("품사 태깅을 지원합니다") 
mecab.morphs("형태소 분리를 지원합니다") 
mecab.nouns("명사에 해당하는 형태소만 추출합니다")

['명사', '해당', '형태소', '추출']

## TF-IDF & 로지스틱 회귀

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
sample_submission = pd.read_csv('dataset/sample_submission.csv')

In [8]:
# tf-idf 벡터화
for max_features in range(100, 10000, 100):
    vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)
    
    X = train['document']
    y = np.array(train.label)
    
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)
    
    X_train = vectorizer.fit_transform(X_train)
    X_eval = vectorizer.transform(X_eval)
    lgs = LogisticRegression(class_weight='balanced')
    lgs.fit(X_train, y_train)
    predicted = lgs.predict(X_eval)
    print(f"max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )

max_features: 100, Accuracy: 0.726
max_features: 200, Accuracy: 0.757
max_features: 300, Accuracy: 0.79
max_features: 400, Accuracy: 0.803
max_features: 500, Accuracy: 0.811
max_features: 600, Accuracy: 0.814
max_features: 700, Accuracy: 0.821
max_features: 800, Accuracy: 0.833
max_features: 900, Accuracy: 0.843
max_features: 1000, Accuracy: 0.844
max_features: 1100, Accuracy: 0.846
max_features: 1200, Accuracy: 0.852
max_features: 1300, Accuracy: 0.849
max_features: 1400, Accuracy: 0.849
max_features: 1500, Accuracy: 0.851
max_features: 1600, Accuracy: 0.847
max_features: 1700, Accuracy: 0.852
max_features: 1800, Accuracy: 0.855
max_features: 1900, Accuracy: 0.859
max_features: 2000, Accuracy: 0.858
max_features: 2100, Accuracy: 0.858
max_features: 2200, Accuracy: 0.858
max_features: 2300, Accuracy: 0.862
max_features: 2400, Accuracy: 0.856
max_features: 2500, Accuracy: 0.859
max_features: 2600, Accuracy: 0.86
max_features: 2700, Accuracy: 0.862
max_features: 2800, Accuracy: 0.864
max

In [53]:
# 4600 일 때 가장 큰 정확도 0.874 를 얻었습니다.
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=9900)

X = train['document']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['document'])
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

X_train = vectorizer.fit_transform(train['document'])
X_test = vectorizer.transform(test['document'])
y = np.array(train.label)

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

AttributeError: 'int' object has no attribute 'lower'

In [11]:
sample_submission = pd.read_csv('dataset/sample_submission.csv')
sample_submission.loc[:, 'label'] = predicted

sample_submission.to_csv('sample_submission_day1_logistic2.csv', index=False)

## 전처리 추가 - 정규표현식
- 각종 특수문자 확인, 한글만 남기고 제거

In [3]:
train.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [4]:
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)

In [5]:
train.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1
2,3,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1
3,4,아무래도 년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,0


In [6]:
test['document'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex = True)

In [7]:
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱
2,3,
3,4,멋있는 영화입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다


## 전처리 추가 - 불용어 제거
- 불용어란 의미가 없는 단어를 의미 

In [41]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

- 한국어 형태소 분석기 중 Okt 형태소 분석기를 사용하여 제거
- Okt(Open Korean Text)는 트위터에서 만든 오픈소스 한국어 처리기인 twitter-korean-text를 이어받아 만들고 있는 프로젝트라고 합니다. 형태소 분석기 중에선 시간이 빠르진 않지만 합리적인 시간을 가지고 있어서 자주 사용된다고 합니다.

In [45]:
from tqdm import tqdm # 진행률 프로세스 바 

In [46]:
from konlpy.tag import Okt

In [47]:
X_train = []
okt = Okt()
for sentence in tqdm(train['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) #토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]#불용어 제거
    X_train.append(stopwords_removed_sentence) # 불용어 제거된 것 append

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:16<00:00, 305.08it/s]


In [48]:
print(X_train[:3])

[['영상', '이나', '음악', '이쁘다', '해도', '미화', '시키다', '불륜', '일', '뿐'], ['히치콕', '영화', '보다', '분명', '박수', '치다'], ['괜찮다', '음악', '영화', '또', '나오다', '따뜻하다', '겨울', '되다', '것', '같다']]


In [49]:
X_test = []
for sentence in tqdm(test['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) #토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]#불용어 제거
    X_train.append(stopwords_removed_sentence) # 불용어 제거된 것 append

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:10<00:00, 469.23it/s]


## 전처리 -  정수 인코딩
- 텍스트를 숫자로 바꾸는 기법 